In [1]:
# import libraries
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [2]:
import sys
sys.path.append("../")
sys.path.append("../../")

import os
import random
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta

from open_data import open_data, create_global_batch
from dataset import RadarDataset,RadarCollate
from simple_model import LSTM
from train import train

In [3]:
PATH_CSV="/Users/clemencevast/Documents/Courses/MachineLearning/RadarTrafficData_files/Radar_Traffic_Counts.csv" #"/Users/iris/Documents/radar_deep/Radar_Traffic_Counts.csv"
radar_name=' CAPITAL OF TEXAS HWY / LAKEWOOD DR'

In [4]:
df1=open_data(PATH_CSV, direction="NB", radar=radar_name, year=2018)

In [5]:
batch_df=create_global_batch(df1, window_x_day=6, window_label_day=1, gap_acquisition=1, tot_len_day=365)

Building batch 0 
 x begin 2018-01-02 label begin 2018-01-08 end period 2018-12-23 
[datetime.date(2018, 1, 2) datetime.date(2018, 1, 3)
 datetime.date(2018, 1, 4) datetime.date(2018, 1, 5)
 datetime.date(2018, 1, 6) datetime.date(2018, 1, 7)]
     index         global_date  Volume  Day of Week        date
96      96 2018-01-02 00:00:00      22            2  2018-01-02
97      97 2018-01-02 00:15:00      13            2  2018-01-02
98      98 2018-01-02 00:30:00      21            2  2018-01-02
99      99 2018-01-02 00:45:00      21            2  2018-01-02
100    100 2018-01-02 01:00:00      15            2  2018-01-02
101    101 2018-01-02 01:15:00      11            2  2018-01-02
102    102 2018-01-02 01:30:00      19            2  2018-01-02
103    103 2018-01-02 01:45:00       5            2  2018-01-02
104    104 2018-01-02 02:00:00      14            2  2018-01-02
105    105 2018-01-02 02:15:00       6            2  2018-01-02
Building batch 1 
 x begin 2018-01-03 label begin 20

In [ ]:
batch_df

In [ ]:
#batch_df["mean_x"]=batch_df["vol_data_x"].apply(lambda x : np.mean(x))
#batch_df["std_x"]=batch_df["vol_data_x"].apply(lambda x : np.std(x))
#batch_df["mean_y"]=batch_df["vol_label_y"].apply(lambda x : np.mean(x))
# batch_df

In [ ]:
def get_df_stats(df,columns=None):
    """ return mean and std for the columns"""
    if columns is None: 
        columns=["vol_data_x","vol_label_y"]
    dic={}
    global_mean=0
    global_std=0
    for name in columns:
        df["{}_min".format(name)]=df[name].apply(lambda x : np.min(x))
        df["{}_max".format(name)]=df[name].apply(lambda x : np.max(x))
        global_mean+=df["{}_min".format(name)].min()
        global_std+=df["{}_max".format(name)].max()
    return global_mean/len(columns),global_std/len(columns)
    
def apply_norm(df,mini,maxi,columns=None):
    """ apply std"""
    if columns is None: 
        columns=["vol_data_x","vol_label_y"]
    for name in columns:
        df["{}_norm".format(name)]=df[name].apply(lambda x : (x-mini)/(maxi-mini))
    return df

In [ ]:
df_mean,df_std=get_df_stats(batch_df)
new_data=apply_norm(batch_df,df_mean,df_std)


In [ ]:
get_df_stats(new_data,["vol_data_x_norm","vol_label_y_norm"])

In [ ]:
new_data

In [ ]:
#get_df_stats(new_data,columns=["vol_data_x_norm","vol_label_y_norm"])

In [ ]:
# sample_rate = 16000
batch_size = 1

In [ ]:
normalize=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[df_mean],
                         std=[df_std])
])
fullset = RadarDataset(dataframe=batch_df,transform=None)

ptrain,pval,ptest=0.8,0.05,0.15
trainglobdataset, testdataset = train_test_split(fullset, test_size=ptest)
traindataset, valdataset = train_test_split(trainglobdataset, test_size=pval/(1-ptest))

In [ ]:
collate_fn = RadarCollate()
trainloader = DataLoader(traindataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
valloader = DataLoader(valdataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
testloader = DataLoader(testdataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

In [ ]:
c= 0
for x in trainloader:
    print("batch number {}".format(c))
    print(x[0].shape, x[1].shape, "\n")
    #print(x)
    c+=1

## Modèle 1 

In [ ]:
input_dim = 576    
hidden_dim = 256
layer_dim = 2
output_dim = 96
seq_dim = 1

lr = 0.05
n_epochs = 100
iterations_per_epoch = len(trainloader)
best_acc = 0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
model=LSTM()


In [ ]:
iteration,loss_train_list,loss_val_list=train(model, trainloader, valloader, lr=lr, n_epochs=n_epochs, device=device,ite_print=5)

In [ ]:
fig,ax=plt.subplots()
ax.plot(iteration,loss_train_list,"b",label="loss train")
ax.plot(iteration,loss_val_list,"r",label="loss validation")
ax.legend()

## Modèle 2 

In [ ]:
input_dim = 576    
hidden_dim = 256
layer_dim = 2
output_dim = 96
seq_dim = 1

lr2 = 0.001
n_epochs = 100
iterations_per_epoch = len(trainloader)
best_acc = 0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model=LSTM()


In [ ]:
iteration2,loss_train_list2,loss_val_list2=train(model, trainloader, valloader, lr=lr2, n_epochs=n_epochs, device=device,ite_print=2)

In [ ]:
fig,ax=plt.subplots()
ax.plot(iteration2,loss_train_list2,"b",label="loss train")
ax.plot(iteration2,loss_val_list2,"r",label="loss validation")
ax.legend()